# 🐍 PyBun クイックスタート 🤖

**PyBun** は、Rustで書かれた **Agent-First Python Runtime** です。

このノートブックでは、PyBunの主要な機能を実際に体験できます：

- 🤖 **AI Native**: `--format=json` でAIエージェントが扱いやすい構造化出力
- ⚡ **高速**: Rustベースの依存解決とモジュールローディング
- 📦 **オールインワン**: pip + venv + test runner が1つのバイナリに
- 🔌 **MCP Server**: Claude Desktop や Cursor との直接統合
- 🛡️ **Sandbox**: 安全なコード実行モード

---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/VOID-TECHNOLOGY-INC/PyBun/blob/main/examples/PyBun_Quick_Start.ipynb)
[![GitHub](https://img.shields.io/badge/GitHub-VOID--TECHNOLOGY--INC%2FPyBun-blue?logo=github)](https://github.com/VOID-TECHNOLOGY-INC/PyBun)
[![PyPI](https://img.shields.io/pypi/v/pybun-cli)](https://pypi.org/project/pybun-cli/)

## 1️⃣ インストール

PyBunは `pip` で簡単にインストールできます。

> **Note**: PATHで `pybun` が [Bun](https://bun.sh) に解決される場合は、`pybun-cli` コマンドを使用してください。

### 💡 Colab / 制限された環境での注意
Google Colabなど、署名検証ツール (`minisign`) がインストールされていない環境では、以下の環境変数を設定してインストールしてください。
```python
%env PYBUN_PYPI_NO_VERIFY=1
```

In [ ]:
# 署名検証をスキップしてインストール（minisignがない環境向け）
%env PYBUN_PYPI_NO_VERIFY=1

# PyBunをインストール
!pip install -q pybun-cli

# バージョン確認
!pybun --version

## 2️⃣ 基本的な使い方

### Pythonスクリプトの実行

`pybun run` で Python スクリプトを実行できます。

In [ ]:
# シンプルなスクリプトを作成
%%writefile hello.py
print("Hello from PyBun! 🐍")
print("This is blazingly fast!")

In [ ]:
# PyBunで実行
!pybun run hello.py

In [ ]:
# プロファイルを指定して実行（prod = Lazy Import最適化有効）
!pybun run --profile=prod hello.py

In [ ]:
# インラインコードも実行可能
!pybun run -c -- "import sys; print(f'Python {sys.version}')"

## 3️⃣ 🤖 AI Nativeな出力 (--format=json)

**これがPyBunの最大の特徴です！**

従来のPythonツールは人間向けのテキスト出力ですが、PyBunは `--format=json` で**AIエージェントが直接パースできる構造化データ**を出力します。

In [ ]:
import json

# JSON形式で実行結果を取得
!pybun --format=json run -c -- "print('Hello, AI!')" > result.json

# 結果をパース
with open('result.json') as f:
    result = json.load(f)
    
print("📋 構造化された実行結果:")
print(json.dumps(result, indent=2, ensure_ascii=False))

In [ ]:
# プログラムからの状態確認
print(f"✅ ステータス: {result.get('status', 'unknown')}")
print(f"📝 コマンド: {result.get('command', 'unknown')}")

### エラーも構造化

エラーが発生しても、AIが理解しやすい形式で情報が提供されます。

`diagnostics` 配列には以下が含まれます：
- `kind`: エラーの種類
- `message`: エラーメッセージ  
- `hint`: **解決方法のヒント** ← これが重要！

In [ ]:
import json
import pprint

# 存在しないファイルを実行（エラーケース）
!pybun --format=json run nonexistent.py > err.json 2>&1 || true

# エラーJSONを整形して表示
try:
    with open('err.json') as f:
        err_result = json.load(f)
    print("🔍 エラーJSONの構造:")
    pprint.pprint(err_result)
except:
    !cat err.json

👆 AIエージェントは正規表現なしでエラーを理解し、`hint` を使って**自動修復**できます！

## 4️⃣ ⚡ 最適化機能（module-find / lazy-import）

PyBunはRustベースの高速なモジュール検索と遅延インポート機能を提供します。

In [ ]:
# Rust製の高速モジュール検索（ベンチマーク付き）
!pybun module-find os.path --benchmark

In [ ]:
# Lazy Import設定の確認
!pybun lazy-import --show-config

In [ ]:
# 特定モジュールのLazy Import判定
!pybun lazy-import --check json

## 5️⃣ 📦 PEP 723: スクリプト埋め込みメタデータ

PyBunは [PEP 723](https://peps.python.org/pep-0723/) をサポートしています。

スクリプト内に依存関係を記述でき、PyBunが**メタデータを解析・表示**します。

> **Note**: 現在は主に**解析と診断**が中心機能です。自動インストール・隔離実行は段階的にリリース予定です。

In [ ]:
%%writefile pep723_demo.py
# /// script
# requires-python = ">=3.9"
# dependencies = [
#   "requests>=2.28.0",
# ]
# ///

"""PEP 723デモ: 依存関係がスクリプト内に埋め込まれています"""

print("✨ This script has embedded metadata!")
print("📦 Dependencies: requests>=2.28.0")
print("🐍 Requires Python: >=3.9")

In [ ]:
# PyBunはこのメタデータを解析できます
!pybun run pep723_demo.py

## 6️⃣ 🛡️ Sandbox モード

AIが生成したコードや信頼できないスクリプトを**安全に実行**できます。

- ファイルシステムアクセスが制限される
- ネットワークアクセスはデフォルトで遮断（`--allow-network` で許可）

In [ ]:
%%writefile sandbox_demo.py
print("🛡️ This code is running in a sandbox!")
print("✅ File system access is restricted")
print("✅ Network access is blocked by default")

In [ ]:
# Sandboxモードで実行
!pybun --format=json run --sandbox sandbox_demo.py

### 🔒 Sandboxの効き目デモ（ネットワーク遮断）

Sandboxモードではネットワークアクセスがブロックされます。

In [ ]:
# ネットワークアクセスがブロックされる例
!pybun --format=json run --sandbox -c -- "import urllib.request as u; u.urlopen('https://example.com').read()" || true

In [ ]:
# --allow-network を付けるとネットワークアクセスが許可される
!pybun --format=json run --sandbox --allow-network -c -- "print('Network access allowed!')"

## 7️⃣ 🔧 環境診断・メンテナンス

### pybun doctor

`pybun doctor` でPython環境の診断ができます。

In [ ]:
# 環境診断（JSON形式）
!pybun --format=json doctor 2>&1 | head -30

### pybun gc（キャッシュ管理）

`pybun gc` でキャッシュのガベージコレクションができます。LRUベースで不要なキャッシュを削除します。

In [ ]:
# キャッシュGC（ドライラン: 実際には削除しない）
!pybun gc --dry-run

In [ ]:
# キャッシュサイズ上限を指定してGC（例: 1GB）
!pybun gc --max-size 1G --dry-run

### 🔒 プライバシー（テレメトリ）

PyBunは**デフォルトでテレメトリを収集しません**（オプトイン設計）。エンタープライズ環境でも安心して使用できます。

In [ ]:
# テレメトリのステータス確認
!pybun telemetry status

## 8️⃣ ⚡ プロファイル設定

PyBunには3つのプロファイルがあります：

| プロファイル | 用途 | 特徴 |
|-------------|------|------|
| `dev` | 開発時 | ホットリロード有効、詳細ログ |
| `prod` | 本番環境 | Lazy Import最適化、静かな出力 |
| `benchmark` | ベンチマーク | 安定したタイミング測定 |

In [ ]:
# 利用可能なプロファイルを確認
!pybun profile --list

In [ ]:
# devプロファイルの設定を確認
!pybun profile dev --show

In [ ]:
# プロファイルの比較
!pybun profile dev --compare prod

## 9️⃣ 🔌 MCP Server（Model Context Protocol）

PyBunは **MCP Server** として動作し、Claude Desktop や Cursor などのAIツールから直接操作できます。

```bash
# stdio モードでMCPサーバーを起動
pybun mcp serve --stdio
```

### 利用可能なMCPツール

| ツール | 説明 |
|--------|------|
| `pybun_resolve` | 依存関係の解決 |
| `pybun_install` | パッケージのインストール |
| `pybun_run` | スクリプトの実行 |
| `pybun_gc` | キャッシュのガベージコレクション |
| `pybun_doctor` | 環境診断 |

### MCP設定例（Claude Desktop）

```json
{
  "mcpServers": {
    "pybun": {
      "command": "pybun",
      "args": ["mcp", "serve", "--stdio"]
    }
  }
}
```

## 🔟 🚀 クイックワンライナー (pybun x)

`pybun x` で一時的にパッケージをインストールして実行できます（`npx` のPython版）。

> **Note**: ネットワーク接続が必要です。

In [ ]:
# cowsayを一時インストールしてバージョンを確認（ネットワーク接続時のみ）
# バージョン指定も可能: pybun x cowsay==6.1 -- --version
!pybun x cowsay -- --version

In [ ]:
# cowsayで挨拶（ネットワーク接続時のみ）
!pybun x cowsay -- -t "Hello from PyBun!"

## 📊 まとめ: なぜPyBunなのか？

| 機能 | 従来ツール (pip, uv, Poetry) | PyBun |
|------|------------------------------|-------|
| **出力形式** | 人間向けテキスト | 🤖 **JSON-first** |
| **AI統合** | 手動パースが必要 | 🔌 **組み込みMCP Server** |
| **エラー処理** | 非構造化メッセージ | 📋 **構造化診断 + ヒント** |
| **Agent自動化** | 脆弱なテキストスクレイピング | ✅ **信頼性の高い機械読み取り可能出力** |
| **安全性** | なし | 🛡️ **Sandbox実行** |
| **モジュール最適化** | なし | ⚡ **Lazy Import / Fast Module Find** |
| **プライバシー** | 不明 | 🔒 **テレメトリ無効がデフォルト** |

---

## 🔗 リンク

- 📘 [GitHub](https://github.com/VOID-TECHNOLOGY-INC/PyBun)
- 📦 [PyPI](https://pypi.org/project/pybun-cli/)
- 📖 [Documentation](https://github.com/VOID-TECHNOLOGY-INC/PyBun/tree/main/docs)

In [ ]:
# 🎉 おめでとうございます！PyBunの基本をマスターしました！
print("""
╔═══════════════════════════════════════════════════════╗
║  🎉 Congratulations! You've learned PyBun basics! 🎉  ║
║                                                       ║
║  Next steps:                                          ║
║  • pip install pybun-cli                              ║
║  • Try it in your projects                            ║
║  • Integrate with Claude Desktop or Cursor            ║
╚═══════════════════════════════════════════════════════╝
""")

---

**Made with ❤️ by [VOID TECHNOLOGY INC](https://github.com/VOID-TECHNOLOGY-INC)**